In [44]:
!pip install yfinance
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
import pandas as pd, numpy as np
import itertools
import seaborn as sns
import yfinance as yf

from copy import copy
import statistics as stats
import math
from functools import reduce

from ta.volume import MFIIndicator
from ta.volatility import AverageTrueRange
from ta.trend import STCIndicator
from ta.trend import EMAIndicator

# Génération du fichier source


In [46]:
ndx = yf.Ticker("^NDX")
df_historical_data = ndx.history(interval="1d", period="max")
df_historical_data.drop(columns=["Dividends","Stock Splits"], inplace=True)
df_historical_data.reset_index(inplace=True)

In [47]:
# Test si aucune ligne manquante
test_list = [champ == 0 for champ in df_historical_data.isnull().sum()]

# Si toutes les colonnes sont True, résultat = True
notnull = all(i for i in test_list)
print("Aucune ligne vide détectée, pour l'ensemble des colonnes : ", notnull)

Aucune ligne vide détectée, pour l'ensemble des colonnes :  True


In [48]:
df_historical_data["Open"] = df_historical_data.Open.apply(lambda x: round(x,2))
df_historical_data["High"] = df_historical_data.High.apply(lambda x: round(x,2))
df_historical_data["Low"] = df_historical_data.Low.apply(lambda x: round(x,2))
df_historical_data["Close"] = df_historical_data.Close.apply(lambda x: round(x,2))

In [49]:
df_historical_data.tail(1)

,Date,Open,High,Low,Close,Volume
9470,2023-04-28 00:00:00-04:00,13139.35,13247.39,13096.94,13245.99,5331380000


In [50]:
df_historical_data.dtypes

Date      datetime64[ns, America/New_York]
Open                               float64
High                               float64
Low                                float64
Close                              float64
Volume                               int64
dtype: object

# Génération des combinaisons de paramètres

In [51]:
l_ema = [e for e in range(40,88,2)]
l_AT_period = [p for p in range(4,42,2)]
l_STC_length = [l for l in range(98,132,2)]
l_STC_slow = [s for s in range(98,132,2)]

In [52]:
all_params = [l_ema, l_AT_period, l_STC_length, l_STC_slow]

In [53]:
combinaisons = list(itertools.product(*all_params))

In [54]:
print("Le nombre de combinaisons possibles est : ",len(combinaisons))

Le nombre de combinaisons possibles est :  131784


# Fonctions

#### Alphatrend

In [55]:
# Trend indicator, équivalent de l'affichage couleur
def trend_indicator(trend):
    if trend > 0 :
        # Uptrend
        x = 1
    elif trend < 0 :
        # Downtrend
        x = -1
    else :
        # Range
        x = 0
    return x

In [56]:
# Defintion fonction
def generate_alphatrend(df_in, mfi_p, mfi_seuil, atr_l, m):
    '''Paramètres d'entrée : longueur MFI, longueur ATR, multiplier
    Retourne les colonnes Alphatrend, Alphatrend +2, Trend (position AT1 / AT2)
    :mfi_p = période MFI servant à délimiter up/down de l'alphatrend
    :mfi_seuil = période MFI pour recherche crossover, détermine uptrend ou downtrend'''

    df = df_in.copy()

    # Colonnes MFI
    s_mfi = MFIIndicator(high=df.High, low=df.Low, close=df.Close, volume=df.Volume, window=mfi_p).money_flow_index()
    df["MFI_ref"] = s_mfi

    # Colonne ATR
    s_atr = AverageTrueRange(high=df.High, low=df.Low, close=df.Close, window=atr_l).average_true_range()
    df["ATR"] = s_atr

    # Lignes UpT et DownT
    df["UpT_support"] = df["Low"] - df["ATR"] * m
    df["DownT_support"] = df["High"] + df["ATR"] * m

    # Suppression des lignes sans signal, en début de DataFrame
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    # ===============================================
    # Calcul Alphatrend, en tant que série
    
    Alphatrend = [0]

    for i in range (1, df.shape[0]):
        # Cas Uptrend
        if df.at[i,"MFI_ref"] >= mfi_seuil :
            if df.at[i,"UpT_support"] < Alphatrend[-1] :
                # Flat
                Alphatrend.append(Alphatrend[-1])
            else :
                # Trailing stop loss Up
                Alphatrend.append(df.at[i,"UpT_support"])

        # Cas Downtrend, MFI < 50
        else :
            if df.at[i,"DownT_support"] > Alphatrend[-1] :
                # Flat
                Alphatrend.append(Alphatrend[-1])
            else :
                # Trailing stop loss Down
                Alphatrend.append(df.at[i,"DownT_support"])

    # ===============================================
    # Ajout des lignes k1 et k2 en tant que colonnes
    
    if df.shape[0] == len(Alphatrend):
        df["Alphatrend_k1"] = pd.Series(Alphatrend).apply(lambda x: round(x,2))
        # Ligne k2 décalée de 2j
        Alphatrend2 = df["Alphatrend_k1"].shift(periods=2, fill_value=0)
        df["Alphatrend_k2"] = pd.Series(Alphatrend2).apply(lambda x: round(x,2))
        # Trend
        df["Trend"] = df.Alphatrend_k1 - df.Alphatrend_k2
        df["Trend"] = df["Trend"].apply(trend_indicator)
    else :
        print("Erreur lors de la génération des lignes Alphatrend")

    # ===============================================
    # Génération des signaux Achat / Vente

    # On isole tous les index non neutres, où AT1 != AT2, à la hausse (1) comme à la baisse (-1)
    s_trend = df["Trend"].loc[df["Trend"]!=0]
    s_trend_diff = s_trend - s_trend.shift(1)

    buy_signal_indexes = s_trend_diff[s_trend_diff == 2].index
    sell_signal_indexes = s_trend_diff[s_trend_diff == -2].index

    df["Signal"] = 0
    df.loc[buy_signal_indexes,"Signal"] = 1
    df.loc[sell_signal_indexes,"Signal"] = -1

    # ===============================================
    # Sélection des colonnes suffisantes
    df = df[["Date","Alphatrend_k1","Alphatrend_k2","Trend","Signal"]]
    
    return df

#### STC & EMA

In [57]:
def generate_STC_and_EMA(df_in, stc_length, fast_length, slow_length, ema_period):
  
  df = df_in[["Date","Close"]].copy()

  s_stc = STCIndicator(close=df.Close, window_slow=slow_length, window_fast=fast_length, cycle=stc_length).stc()
  s_ema = EMAIndicator(close=df.Close, window=ema_period).ema_indicator()

  df["STC"] = round(s_stc,2)
  df["EMA"] = round(s_ema,2)

  df.drop(columns=["Close"], inplace=True)

  return df

#### ATR sortie & Merge tous indicateurs techniques

In [58]:
def merge_technical_indicators(df_in, atr_l, df1, df2, date_min="1998-01-01"):
  ''' Fusionne les DataFrames d'indicateurs techniques, 
  ajoute également date_min au format 'yyyy-mm-dd' pour fixer le début du Backtesting'''

  df_essentials = df_in.copy()

  # Ajout de la colonne ATR qui servira plus tard dans le calcul de la sortie.
  s_atr = AverageTrueRange(high=df_essentials.High, low=df_essentials.Low, close=df_essentials.Close, window=atr_l).average_true_range()
  df_essentials["ATR"] = pd.Series(s_atr).apply(lambda x: round(x,2))
  
  # Réduction au strict nécessaire pour les colonnes
  df_essentials = df_essentials[["Date","Open","Close","ATR"]].copy()

  # Merge des 3
  data_frames = [df_essentials, df1, df2]
  df_merged = reduce(lambda  left,right: pd.merge(left,right, on=['Date'], how='left'), data_frames)

  # Réduction de la fenêtre de tests à partir de la date_min
  df_merged = df_merged.loc[df_merged["Date"] >= date_min]

  df_merged.reset_index(drop=True,inplace=True)

  return df_merged


#### Détection des entrées

Stratégie :<br>
<li>Entreée : Buy signal + Prix > EMA + STC < seuil(25)</li>
<li>Sortie : Sell signal + Prix < EMA + STC > seuil(75)</li>

In [59]:
def get_entries_signals(df_in, stc_low, stc_high):
  ''' Nécessite en entrée le DataFrame avec indicateurs techniques.
  Génère aussi bien signaux Entrée Long (1) que Entrée Short (-1).'''

  df_IT = df_in.copy()
  stc_seuil_bas = stc_low
  stc_seuil_haut = stc_high

  # Valeur 3 pour signaux d'entrée valides
  df_IT["Buy_entry"] = np.sign(df_IT.Close - df_IT.EMA) + df_IT.Signal + np.sign(stc_seuil_bas - df_IT.STC)
  # Valeur -3 pour signaux d'entrée valides
  # Attention / par deux signaux négatifs -> positif, d'où l'inversion sur un seul champ
  df_IT["Sell_entry"] = np.sign(df_IT.Close - df_IT.EMA) + df_IT.Signal + np.sign(stc_seuil_haut - df_IT.STC)

  # Conversion en np array
  arr_buy_entry = df_IT["Buy_entry"].to_numpy()
  # np.where(condition, vrai, sinon)
  df_IT["Buy_entry"] = np.where(arr_buy_entry==3, 1, 0)

  arr_sell_entry = df_IT["Sell_entry"].to_numpy()
  df_IT["Sell_entry"] = np.where(arr_sell_entry==-3.0, -1, 0)
  
  # Agrégation des deux types de signaux.
  df_IT["Entry"] = df_IT["Sell_entry"] + df_IT["Buy_entry"]
  df_IT.drop(columns=["Buy_entry","Sell_entry"], inplace=True)

# Strategy as Class

In [60]:
class Strat_AT_STC_EMA:
  
  def __init__(self, p_ema=200, p_AT_m=1, p_AT_l=14, p_AT_mfi_l = 14, p_AT_mfi_s = 50, p_STC_l=80, p_STC_slow_l=50, p_STC_fast_l=27, p_STC_b=25, p_STC_h=75, p_ATR_SL_l = 14, p_ATR_SL = 2, p_RR_ratio = 3):
    self.ema_l = p_ema
    self.at_m = p_AT_m
    self.at_l = p_AT_l
    self.at_mfi_l = p_AT_mfi_l
    self.at_mfi_s = p_AT_mfi_s
    self.stc_l = p_STC_l
    self.stc_s_l = p_STC_slow_l
    self.stc_f_l = p_STC_fast_l
    self.stc_seuil_b = p_STC_b
    self.stc_seuil_h = p_STC_h
    self.ATR_SL_l = p_ATR_SL_l
    self.ATR_SL = p_ATR_SL
    self.RR_ratio = p_RR_ratio

  
  def make_technical_indicators(self, df_source):
    df_AT = generate_alphatrend(df_source, mfi_p=self.at_mfi_l, mfi_seuil=self.at_mfi_s, atr_l=self.at_l, m=self.at_m)
    df_STC_EMA = generate_STC_and_EMA(df_source, stc_length=self.stc_l, fast_length=self.stc_f_l, slow_length=self.stc_s_l, ema_period=self.ema_l)
    df_Technical_Indicators = merge_technical_indicators(df_source, self.ATR_SL_l, df_AT, df_STC_EMA)
    return df_Technical_Indicators

  

# Test

In [61]:
x = Strat_AT_STC_EMA()

In [62]:
%time
df_test = x.make_technical_indicators(df_historical_data)
df_test

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.54 µs


,Date,Open,Close,ATR,Alphatrend_k1,Alphatrend_k2,Trend,Signal,STC,EMA
0,1998-01-02 00:00:00-05:00,990.80,1008.23,24.18,979.81,979.81,0.0,0.0,5.72,998.30
1,1998-01-05 00:00:00-05:00,1008.23,1017.42,23.98,983.27,979.81,1.0,1.0,11.67,998.49
2,1998-01-06 00:00:00-05:00,1017.42,1006.29,23.43,983.27,979.81,1.0,0.0,21.32,998.56
3,1998-01-07 00:00:00-05:00,1006.29,991.19,23.73,983.27,983.27,0.0,0.0,34.99,998.49
4,1998-01-08 00:00:00-05:00,990.97,994.55,23.56,983.27,983.27,0.0,0.0,45.28,998.45
...,...,...,...,...,...,...,...,...,...,...
6367,2023-04-24 00:00:00-04:00,12981.71,12969.76,192.90,12837.07,12837.07,0.0,0.0,95.98,12341.02
6368,2023-04-25 00:00:00-04:00,12905.09,12725.11,196.66,12837.07,12837.07,0.0,0.0,92.51,12344.84
6369,2023-04-26 00:00:00-04:00,12866.64,12806.48,197.22,12837.07,12837.07,0.0,0.0,88.08,12349.44
6370,2023-04-27 00:00:00-04:00,12963.21,13160.03,209.50,12837.07,12837.07,0.0,0.0,84.80,12357.50
